# Imports

In [123]:
import pandas as pd
import matplotlib.pyplot as plt
from ann_visualizer.visualize import ann_viz;
import numpy as np

from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

In [92]:
from warnings import filterwarnings
filterwarnings('ignore')

# Loads Data

In [93]:
df = pd.read_csv(r"League_Result_Data/Encoded_PremierLeague_Stats_From_2014to2021.csv")
df

,Unnamed: 0,Season,Season Encoding,Date,YearOfSeason,HomeTeam,HomeTeam Encoding,AwayTeam,AwayTeam Encoding,FTHG,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,0,Fall,0,12/09/2020,2020/21,Fulham,8,Arsenal,0,0.0,...,2.0,6.0,12.0,12.0,2.0,3.0,2.0,2.0,0.0,0.0
1,1,Fall,0,12/09/2020,2020/21,Crystal Palace,6,Southampton,21,1.0,...,3.0,5.0,14.0,11.0,7.0,3.0,2.0,1.0,0.0,0.0
2,2,Fall,0,12/09/2020,2020/21,Liverpool,13,Leeds,11,4.0,...,6.0,3.0,9.0,6.0,9.0,0.0,1.0,0.0,0.0,0.0
3,3,Fall,0,12/09/2020,2020/21,West Ham,28,Newcastle,17,0.0,...,3.0,2.0,13.0,7.0,8.0,7.0,2.0,2.0,0.0,0.0
4,4,Fall,0,13/09/2020,2020/21,West Brom,27,Leicester,12,0.0,...,1.0,7.0,12.0,9.0,2.0,5.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564,2564,Spring,2,24/05/15,2014/15,Everton,7,Tottenham,25,0.0,...,1.0,3.0,12.0,8.0,3.0,5.0,1.0,2.0,0.0,0.0
2565,2565,Spring,2,24/05/15,2014/15,Hull,10,Man United,15,0.0,...,6.0,1.0,12.0,15.0,8.0,1.0,2.0,2.0,0.0,1.0
2566,2566,Spring,2,24/05/15,2014/15,Leicester,12,QPR,19,5.0,...,7.0,2.0,7.0,6.0,5.0,6.0,0.0,0.0,0.0,0.0
2567,2567,Spring,2,24/05/15,2014/15,Man City,14,Southampton,21,2.0,...,6.0,4.0,13.0,8.0,8.0,4.0,1.0,1.0,0.0,0.0


In [94]:
# Features -- Drops FTR and any categorical value 
X = df.drop(columns=["Season", "Unnamed: 0", "YearOfSeason", "Date", "FTHG", "FTAG", "HTHG", "HTAG", "HomeTeam", "AwayTeam", "Referee", "FTR", "FTR Encoding", "HTR", "Referee"])
# Labels
y = df["FTR Encoding"]

X

,Season Encoding,HomeTeam Encoding,AwayTeam Encoding,HTR Encoding,Referee Encoding,Fouls Called Per Game,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,0,8,0,1,7,14.0,5.0,13.0,2.0,6.0,12.0,12.0,2.0,3.0,2.0,2.0,0.0,0.0
1,0,6,21,0,27,14.0,5.0,9.0,3.0,5.0,14.0,11.0,7.0,3.0,2.0,1.0,0.0,0.0
2,0,13,11,0,28,14.0,22.0,6.0,6.0,3.0,9.0,6.0,9.0,0.0,1.0,0.0,0.0,0.0
3,0,28,17,2,11,13.0,15.0,15.0,3.0,2.0,13.0,7.0,8.0,7.0,2.0,2.0,0.0,0.0
4,0,27,12,2,10,15.0,7.0,13.0,1.0,7.0,12.0,9.0,2.0,5.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2564,2,7,25,1,27,14.0,9.0,16.0,1.0,3.0,12.0,8.0,3.0,5.0,1.0,2.0,0.0,0.0
2565,2,10,15,2,15,13.0,16.0,7.0,6.0,1.0,12.0,15.0,8.0,1.0,2.0,2.0,0.0,1.0
2566,2,12,19,0,28,14.0,22.0,18.0,7.0,2.0,7.0,6.0,5.0,6.0,0.0,0.0,0.0,0.0
2567,2,14,21,0,2,14.0,15.0,13.0,6.0,4.0,13.0,8.0,8.0,4.0,1.0,1.0,0.0,0.0


In [95]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2569 entries, 0 to 2568
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Season Encoding        2569 non-null   int64  
 1   HomeTeam Encoding      2569 non-null   int64  
 2   AwayTeam Encoding      2569 non-null   int64  
 3   HTR Encoding           2569 non-null   int64  
 4   Referee Encoding       2569 non-null   int64  
 5   Fouls Called Per Game  2569 non-null   float64
 6   HS                     2569 non-null   float64
 7   AS                     2569 non-null   float64
 8   HST                    2569 non-null   float64
 9   AST                    2569 non-null   float64
 10  HF                     2569 non-null   float64
 11  AF                     2569 non-null   float64
 12  HC                     2569 non-null   float64
 13  AC                     2569 non-null   float64
 14  HY                     2569 non-null   float64
 15  AY  

In [96]:
"""
Target Varible - Full Time Results
0 - Home Team Win
1 - Away Team Win
2 - Draw
"""

y

0       1
1       0
2       0
3       1
4       1
       ..
2564    1
2565    2
2566    0
2567    0
2568    0
Name: FTR Encoding, Length: 2569, dtype: int64

# Split the Data

In [97]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

# Scale the Data

In [98]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# Fitting & Evaluating the Model

In [142]:
nn_clf = MLPClassifier(solver='adam', alpha=1e-3,
                     hidden_layer_sizes=(5, 4), random_state=42,
                       max_iter=500, learning_rate='adaptive',
                       activation='logistic',
                       power_t=0.75,
                       warm_start=False,
                       beta_2=0.75,
                       beta_1=0.95
                      )

nn_clf.fit(X_train, y_train)

score = nn_clf.score(X_test, y_test)

print(score)

0.6517509727626459


In [143]:
results = nn_clf.predict(X_test)

In [144]:
# Precision_Recall_F-Score
p_r_f = precision_recall_fscore_support(y_test, results, average='macro')
p_r_f

(0.5867182462927144, 0.5927719798738504, 0.5628841379908408, None)

In [145]:
test_acc = accuracy_score(y_test, results)
test_acc

0.6517509727626459

In [146]:
cv_dict = cross_validate(nn_clf, X, y, return_train_score=True)
cv_dict

{'fit_time': array([2.36199927, 1.35303068, 1.95902467, 1.94002581, 2.08413243]),
 'score_time': array([0.00202656, 0.00199986, 0.002002  , 0.0010066 , 0.00299931]),
 'test_score': array([0.64980545, 0.60116732, 0.61673152, 0.61478599, 0.582846  ]),
 'train_score': array([0.64476886, 0.62384428, 0.6107056 , 0.6136253 , 0.62208171])}

In [147]:
# Confusion Matrix

cv_score = cross_val_score(nn_clf, X_train, y_train, cv=3, scoring='accuracy')
print('Accuracy:',cv_score, '\n')

y_train_pred = cross_val_predict(nn_clf, X_train, y_train, cv=3)
print(confusion_matrix(y_train, y_train_pred))

Accuracy: [0.64525547 0.63357664 0.64671533] 

[[812 110   3]
 [138 505   2]
 [264 219   2]]


# Evaluating the Model

In [148]:
nn_clf.classes_

array([0, 1, 2], dtype=int64)

In [149]:
nn_clf.loss_

0.7185753173796928

In [150]:
nn_clf.best_loss_

0.7185753173796928

In [151]:
nn_clf.loss_curve_

[1.2007826026585213,
 1.1841278894056586,
 1.169013311031387,
 1.1536767147755975,
 1.1394024800689249,
 1.1265356787729215,
 1.1132869870769522,
 1.1030229428063067,
 1.093336738687993,
 1.0847607896700124,
 1.076300383175746,
 1.070701845153319,
 1.0651581259835263,
 1.0617057518169777,
 1.0589810078868336,
 1.0574954513566348,
 1.0558426467163322,
 1.0545254100800063,
 1.0532595705829861,
 1.0520024225556166,
 1.0507279275314878,
 1.0493265766674928,
 1.0478402958986917,
 1.046262800996168,
 1.0446655562954796,
 1.0428785465576105,
 1.0409931041478226,
 1.0391409081034462,
 1.0371105471478874,
 1.0350286902813557,
 1.0328229797688708,
 1.030572773986635,
 1.0280835352206297,
 1.0256451423150499,
 1.0231259252509928,
 1.0205697536639968,
 1.0179357922574603,
 1.0153912804543632,
 1.0130529985518435,
 1.0103998357333135,
 1.0077824141953793,
 1.0051380651352169,
 1.0024718317430936,
 0.999458040376841,
 0.9966771673145985,
 0.9938702711945516,
 0.9910847956331619,
 0.9883101740946229,

In [152]:
str(nn_clf.t_) + " training instances seen during fitting"

'1027500 training instances seen during fitting'

In [153]:
str(nn_clf.n_iter_) + "  iterations"

'500  iterations'

In [154]:
str(nn_clf.n_layers_) + " layers"

'4 layers'

In [155]:
str(nn_clf.n_outputs_) + " outputs"

'3 outputs'

In [156]:
"Output Activation: " + str(nn_clf.out_activation_)

'Output Activation: softmax'

In [157]:
nn_clf.get_params(nn_clf)

{'activation': 'logistic',
 'alpha': 0.001,
 'batch_size': 'auto',
 'beta_1': 0.95,
 'beta_2': 0.75,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (5, 4),
 'learning_rate': 'adaptive',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 500,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.75,
 'random_state': 42,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [158]:
nn_clf.coefs_

[array([[ 5.72750626e-03,  4.38443354e-01,  2.26791515e-01,
         -3.61576035e-01, -5.28974201e-01],
        [-1.62284782e-01, -4.17028537e-02,  2.83335862e-01,
         -8.89629137e-02,  8.53327223e-02],
        [ 5.12634700e-02,  5.59312862e-01, -1.94705114e-01,
         -1.90388878e-01, -1.71772670e-01],
        [-3.49555925e+00,  1.34939998e+00,  9.09144685e-01,
          1.42494247e+00,  7.41374059e-01],
        [-8.06629472e-02,  2.01247407e-02, -2.02121103e-04,
          1.23096063e-01,  1.97843411e-01],
        [ 9.37366974e-02,  3.22782147e-01, -6.61218905e-01,
          3.46045745e-01, -2.00569312e-01],
        [-2.20929170e-01, -4.18609735e-01, -9.78660448e-02,
          1.47287264e-01,  4.27427572e-01],
        [ 3.86242126e-02, -1.90681556e-01, -8.63477041e-03,
          2.42193919e-01, -1.26292383e-01],
        [ 4.07145281e-01,  4.88700079e-01,  6.55733874e-01,
          1.29308954e+00,  5.55076404e-01],
        [-3.54646806e-01, -1.01754480e+00, -7.45665704e-01,
    

In [159]:
len(nn_clf.coefs_)

3

In [160]:
len(nn_clf.coefs_[0])

18

In [161]:
len(nn_clf.coefs_[1])

5

In [162]:
len(nn_clf.coefs_[2])

4

# Test With a Recent Game

Game Used: Man City vs Chelsea FA Cup (4/17/2021) -- Chelsea won (ie the Home Team won)

Data came from these links and our data
- https://www.espn.com/soccer/matchstats?gameId=597190
- https://www.si.com/soccer/manchestercity/match-coverage/manchester-city-vs-chelsea-where-to-watch-team-news-referees-everything-you-need-to-know

In [163]:
list(X.columns) # The values we need to make a prediction 

['Season Encoding',
 'HomeTeam Encoding',
 'AwayTeam Encoding',
 'HTR Encoding',
 'Referee Encoding',
 'Fouls Called Per Game',
 'HS',
 'AS',
 'HST',
 'AST',
 'HF',
 'AF',
 'HC',
 'AC',
 'HY',
 'AY',
 'HR',
 'AR']

In [164]:
# Data came from these links and our data

test_game = np.array([
    2, # 2 bc it took place in Spring
    0, # 0 bc kickoff was at 12:32 (rounds to 12:30)
    5, # Chelsea: 5 in the Team encoding
    14, # Man City: 14 in the Team encoding
    #0,
    #0,
    23, # Mike Dean was the referee
    14, # got from our data
    5,
    11,
    3,
    3,
    8,
    12,
    3,
    8,
    2,
    3,
    0,
    0
            ]).reshape(1, -1)

result = nn_clf.predict(test_game)
print(result)

if result[0] == 0:
    print("Correct!",end=' ')
else:
    print("Incorrect!",end=' ')
    
print("Chelsea, the Home Team, won.")

[0]
Correct! Chelsea, the Home Team, won.
